In [ ]:
# !apt-get update 
# !pip3 install pybullet
# !apt-get install -y ffmpeg
# https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#

In [ ]:
import torch

import pybullet as p
import time
import numpy as np
import os
import sys
from collections import namedtuple

import _utils_v2 as utils
from _utils_v2 import geom_shape, xyz_v2, s, CFG_Loader, rad2deg, deg2rad

from builder import initializer, builder, main
# from body_move import move


class daughter:
    def __init__(self, 
                 path2cfgs, 
                 VARIABLE        = 0.04,
                 fig_joint_coeff = 0.25,
                 centered        = False):
        # super(daughter, self).__init__()
        
        self.path2cfgs = path2cfgs
        self.VARIABLE  = VARIABLE
        self.joint_cf  = fig_joint_coeff
        self.centered  = centered 
        self.body_Mass = 200 if centered else 1e-5
        self.position  = utils.coordinates(0,0,0) if centered else utils.coordinates(0,0,1)
        self.init_cfgs = initializer(self.path2cfgs, self.VARIABLE, self.joint_cf)
        
    def run_enviroment(self, mode = p.GUI):
        p.connect(mode)# Avalible p.GUI p.DIRECT
        p.createCollisionShape(p.GEOM_PLANE, planeNormal = [0,0,1])
        p.createMultiBody(0,0)
        
        self.blocks = builder(self.init_cfgs, self.position)
        self.body   = main(self.blocks, self.body_Mass, self.centered)
        self.minmax = [data.angles for data in self.body.cfgs if data.jntTypes==0]
        self.mtrIdx = [i for i, data in enumerate(self.body.cfgs) if data.jntTypes==0]
        self.names  = [(i, data.Name) for i, data in enumerate(self.body.cfgs) if data.jntTypes==0]
        
        self.force  = [data.torque for data in self.body.cfgs if data.jntTypes==0]
        self.velocity=[data.ang_speed for data in self.body.cfgs if data.jntTypes==0]
        
        self.body.create_body()
        
    def stop_enviroment(self):
        p.disconnect()   
    
    def reset(self):
        p.setGravity(0,0,0)
        
        first_state = [0]*len(self.mtrIdx)
        utils.resetCoordinates(1, self.mtrIdx, first_state, self.force, [i * 100 for i in self.velocity]) 
        
        time.sleep(1)
        p.resetBasePositionAndOrientation(1, [0, 0, 1.1], [0, 0, 0, 1])
        p.setGravity(0,0,-9.81)
        
        return first_state
    
    def step(self, action):
        next_state = 0
        # utils.resetCoordinates(1, self.mtrIdx, next_state, self.force, self.velocity)
        
        base_loss = 0
        reward = base_loss  
         
        b_position, _ = p.getBasePositionAndOrientation(1)
        game_over = False if b_position[2] < 0.6 else True
        additional_info = b_position
        return next_state, reward, game_over, additional_info
    
    
class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.hw = 'Hello, World!'
        self.Linear_1 = torch.nn.Linear(26 ,  52)
        self.Linear_2 = torch.nn.Linear(52 , 104)
        self.Linear_3 = torch.nn.Linear(104, 208)
        self.Linear_4 = torch.nn.Linear(208, 104)
        self.Linear_5 = torch.nn.Linear(104,  52)
        self.classifier = torch.nn.Linear(52 ,26)
        self.regressor  = torch.nn.Linear(52 ,26)
        
    def forward(self, x):
        x = self.Linear_1(x)
        x = self.Linear_2(x)
        x = self.Linear_3(x)
        x = self.Linear_4(x)
        x = self.Linear_5(x)
        cls = self.classifier(x)
        rgr = self.regressor(x)
        return cls, rgr
    
def optimizer_init(net, lr  = 1e-3,  momentum = 0.9, 
                   wght_dcy = 5e-4,  epsilon  = 1e-8, beta1 = 0.9,  beta2 = 0.999, 
                   amsgrad  = False, is_SGD   = True, custom_params = True):
    if custom_params:
        biases, not_biases = list(), list()
        for param_name, param in net.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        params = [{'params': biases, 'lr': 2*lr}, {'params': not_biases}]
    else:
        params = net.parameters()
    if is_SGD:
        optimizer = torch.optim.SGD(
            params       = params,
            lr           = lr,
            momentum     = momentum,
            weight_decay = wght_dcy)
    else:
        optimizer = torch.optim.Adam(
            params       = params, 
            lr           = lr, 
            betas        = (beta1,beta2), 
            eps          = epsilon,
            weight_decay = wght_dcy, 
            amsgrad      = amsgrad)
    return optimizer

In [ ]:
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F

path = './cfgs'
enviroment = daughter(path)
enviroment.run_enviroment()
angles = enviroment.reset()

# Net initialization
net = model()
# Optimizer initialization 
optimizer = optimizer_init(net)

In [ ]:
def searcher(pathOfName, enviroment_names):
    rez = [(i) for i in enviroment_names 
           if pathOfName in i[1].Name]
    return rez

In [ ]:
angles = enviroment.reset()

In [ ]:
enviroment.stop_enviroment()

In [ ]:
motors = [(i,j) for i, j in enumerate(enviroment.body.cfgs) if j.jntTypes==0]

In [ ]:
p.setGravity(0,0,-9.81)
name = 'ShoulderF'
motor_indexes = [i[0] for i in searcher(name, motors)]
forces = [i[1].torque for i in searcher(name, motors)]
velocities = [i[1].ang_speed for i in searcher(name, motors)]
angles = [
    -190,-190,
] 
utils.resetCoordinates(1, motor_indexes, angles, forces, velocities)
print([i for i in enviroment.names if i[0] in motor_indexes])

In [ ]:
while (1):
    pos, ori = p.getBasePositionAndOrientation(1)
    if pos[2] < 0.5:
        angles = enviroment.reset()
        break

In [ ]:
# # jointPosition (in radians along avalible axis)
# # jointVelocity
# # jointReactionForces
# # appliedJointMotorTorque

# data                    = p.getJointStates(1,motor_indexes)
# jointPosition           = [el[0] for el in data]
# jointVelocity           = [el[1] for el in data]
# jointReactionForces     = [el[2] for el in data]
# appliedJointMotorTorque = [el[3] for el in data]

# angles = [rad2deg(rad) for rad in jointPosition]
# angles

In [ ]:
action,state = net(state/360)
state = state*360
condition = True

In [ ]:
action,state

In [ ]:
# values
# indices[-values[values>1e-5].shape[0]]

In [ ]:
mts = values[values>1e-2].shape[0]
mtrIdx = indices[-mts:]
next_state = state[indices[-mts:]]

utils.resetCoordinates(1, mtrIdx, next_state)

In [ ]:
#with torch.autograd.set_detect_anomaly(True):
optimizer.zero_grad()
r.backward()
optimizer.step()

In [ ]:
next_state, overCharge = self.clamp(action)
utils.resetCoordinates(1, self.mtrIdx, next_state)
base_loss = torch.tensor(data = (100 - (time.time() - self.global_timer)), 
                 dtype = torch.float16)
reward = base_loss # sum(overCharge) + 

b_position, _ = p.getBasePositionAndOrientation(1)
game_over = False if b_position[2] < 0.6 else True
additional_info = b_position
# return next_state, reward, game_over, additional_info

In [ ]:
#         action, reward, \
#         condition,      \
#         additional_info = enviroment.step(action)

#         optimizer.zero_grad()
#         reward.backward()
#         optimizer.step()

In [ ]:
angles = enviroment.reset()
state  = torch.tensor(angles, dtype = torch.float32)

In [ ]:
action, state = net(state/360)
state = state*360
action, state

In [ ]:
rez = F.softmax(action)
values, indices = rez.sort()
values, indices

In [ ]:
for epoch in tqdm(range(100)):
    
    angles = enviroment.reset()
    action = torch.tensor(angles, dtype = torch.float32)
    start_time = time.time()
    condition = True
    
    while (1):
        pos, ori = p.getBasePositionAndOrientation(1)
        if pos[2] < 0.5:
            break
        action, state = net(state/360)
        state = state * 360
        
        time.sleep(1)

# next_state = 0
# utils.resetCoordinates(1, self.mtrIdx, next_state, self.force, self.velocity)

# base_loss = 0
# reward = base_loss  

# b_position, _ = p.getBasePositionAndOrientation(1)
# game_over = False if b_position[2] < 0.6 else True
# additional_info = b_position
# # next_state, reward, game_over, additional_info

In [ ]:
enviroment.stop_enviroment()